In [1]:
%%bash
wget --no-check-certificate 'https://drive.usercontent.google.com/download?id=1qe5hOSBxzIuxBb1G_Ih5X-O65QElollE&export=download&confirm=t&uuid=b2002093-cc6e-4bd5-be47-9603f0b33470
' -O KuaiRec.zip
unzip KuaiRec.zip -d data_final_project

--2025-04-14 11:29:41--  https://drive.usercontent.google.com/download?id=1qe5hOSBxzIuxBb1G_Ih5X-O65QElollE&export=download&confirm=t&uuid=b2002093-cc6e-4bd5-be47-9603f0b33470%0A


Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.179.65, 2a00:1450:4007:813::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.179.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 431964858 (412M) [application/octet-stream]
Saving to: ‘KuaiRec.zip’

     0K .......... .......... .......... .......... ..........  0% 3.70M 1m51s
    50K .......... .......... .......... .......... ..........  0% 12.7M 72s
   100K .......... .......... .......... .......... ..........  0% 9.69M 62s
   150K .......... .......... .......... .......... ..........  0% 18.1M 52s
   200K .......... .......... .......... .......... ..........  0% 9.05M 51s
   250K .......... .......... .......... .......... ..........  0% 32.2M 44s
   300K .......... .......... .......... .......... ..........  0% 7.62M 46s
   350K .......... .......... .......... .......... ..........  0% 11.0M 45s
   400K .......... .......... .....

Archive:  KuaiRec.zip
   creating: data_final_project/KuaiRec 2.0/
  inflating: data_final_project/KuaiRec 2.0/LICENSE  
  inflating: data_final_project/KuaiRec 2.0/Statistics_KuaiRec.ipynb  
   creating: data_final_project/KuaiRec 2.0/data/
  inflating: data_final_project/KuaiRec 2.0/data/big_matrix.csv  
  inflating: data_final_project/KuaiRec 2.0/data/item_categories.csv  
  inflating: data_final_project/KuaiRec 2.0/data/item_daily_features.csv  
  inflating: data_final_project/KuaiRec 2.0/data/kuairec_caption_category.csv  
  inflating: data_final_project/KuaiRec 2.0/data/small_matrix.csv  
  inflating: data_final_project/KuaiRec 2.0/data/social_network.csv  
  inflating: data_final_project/KuaiRec 2.0/data/user_features.csv  
   creating: data_final_project/KuaiRec 2.0/figs/
  inflating: data_final_project/KuaiRec 2.0/figs/KuaiRec.png  
  inflating: data_final_project/KuaiRec 2.0/figs/colab-badge.svg  
  inflating: data_final_project/KuaiRec 2.0/loaddata.py  


In [4]:
# Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import ast

2025-04-20 22:40:43.104225: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [30]:
# Drop every time related columns except "timestamp" because of redundancy (keeping timestamp because it is UTC). Same for the play time and video time
train_set = pd.read_csv("data_final_project/KuaiRec 2.0/data/big_matrix.csv", usecols=["user_id", "video_id", "watch_ratio"]).reset_index(drop=True)
test_set = pd.read_csv("data_final_project/KuaiRec 2.0/data/small_matrix.csv", usecols=["user_id", "video_id", "watch_ratio"]).reset_index(drop=True)

tags = pd.read_csv("data_final_project/KuaiRec 2.0/data/item_categories.csv").reset_index(drop=True)

In [31]:
tags["feat"] = tags["feat"].apply(ast.literal_eval)

In [32]:
def clean_df(df: pd.DataFrame):
    df = df.dropna()
    df = df.drop_duplicates()
    #df = df[df["timestamp"] >= 0]
    df = df.drop(df[df["watch_ratio"] == 0].index)
    return df

train_set = clean_df(train_set)
test_set = clean_df(test_set)

In [ ]:
def build_feat_stats():
    tags_data = tags['feat'].explode().value_counts().sort_index().reset_index()
    tags_data.columns = ['feat', 'nb_videos']

    tmp_set = train_set.drop(columns=["user_id"])
    tmp_set = tmp_set.merge(tags, on="video_id").explode("feat")
    nb_ratings = tmp_set['feat'].value_counts()
    nb_ratings = nb_ratings.sort_index().reset_index()
    nb_ratings.columns = ["feat", "nb_ratings"]
    tmp_set = tmp_set.groupby("feat")["watch_ratio"].mean()

    tmp_set = tmp_set.reset_index()
    tmp_set.columns = ["feat", "avg_watch_ratio"]
    
    tags_data = tags_data.merge(tmp_set, on="feat")
    tags_data = tags_data.merge(nb_ratings, on="feat")
    
    return tags_data


In [43]:
def build_users_stats():
    rating_count = train_set.groupby('user_id').size().sort_index().reset_index(name='nb_ratings')
    avg_rating = train_set.groupby('user_id')["watch_ratio"].mean().reset_index(name='avg_rating')
    
    users_data = train_set.merge(tags, on="video_id").explode("feat")
    users_data = users_data.groupby(['user_id', 'feat'])['watch_ratio'].mean().reset_index()
    users_data = users_data.pivot(index='user_id', columns='feat', values='watch_ratio').fillna(0)
    users_data.columns = [f'feat_{int(col)}' for col in users_data.columns]
    users_data = users_data.merge(rating_count, on='user_id')
    users_data = users_data.merge(avg_rating, on='user_id')
    return users_data

In [35]:
build_feat_stats()

,feat,nb_videos,avg_watch_ratio,nb_ratings
0,0,24,0.874633,42663
1,1,463,0.955795,531193
2,2,216,0.758936,111796
3,3,104,0.718970,43361
4,4,164,0.949445,136872
5,5,964,1.021739,475621
6,6,912,0.775626,1429086
7,7,503,1.061024,718532
8,8,1112,1.001294,1657114
9,9,696,0.910919,1064911


In [44]:
build_users_stats()

,user_id,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,...,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,nb_ratings,avg_rating
0,0,0.845523,0.942779,0.877975,1.027538,0.716372,0.996952,0.869039,1.081366,1.065330,...,1.172116,2.183256,1.254409,1.370651,0.640866,1.118369,1.032785,0.467586,2374,1.108412
1,1,0.365590,0.703483,0.220153,0.114642,1.644048,0.777455,0.903273,0.977473,0.897767,...,0.198339,0.000000,0.710297,1.042393,2.134150,1.125668,1.434024,0.951391,1377,0.989412
2,2,0.376160,0.601405,0.370203,0.558891,0.880116,0.622800,0.492298,0.771608,0.723766,...,0.771147,0.108416,0.678943,0.759984,0.241906,0.654655,0.176768,0.089393,1016,0.655095
3,3,1.203877,0.901088,0.743963,0.796545,0.989450,1.073212,0.728408,1.017462,0.943273,...,1.113458,2.083893,0.914847,0.752743,0.526340,1.081395,1.252024,0.419402,3030,0.930452
4,4,0.612527,0.748950,1.010854,0.000000,0.455023,0.271693,0.474947,0.844550,0.695059,...,0.000000,0.000000,1.037390,0.898301,0.128316,0.861795,0.214988,0.000000,541,0.812937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7171,7171,0.522953,0.974668,0.673035,0.526626,0.702024,0.810527,0.613793,1.328922,0.674029,...,0.416539,0.064668,1.268646,0.804917,0.846472,1.206591,0.181914,0.511466,1290,0.965447
7172,7172,0.712095,1.346675,0.738943,0.341281,0.863731,0.958170,0.988892,1.075962,1.366801,...,1.305065,0.081499,1.059291,1.170765,0.797202,1.435807,0.000000,0.496231,2050,1.118402
7173,7173,0.293560,0.397053,1.181011,0.000000,1.667356,0.391223,0.646296,0.600301,0.509637,...,0.441495,0.000000,1.031987,1.157426,0.000000,1.112405,0.000000,1.531496,799,1.119829
7174,7174,1.071403,0.697977,0.556874,0.671852,0.907333,0.799350,0.573484,0.835290,0.753470,...,2.163089,0.122090,0.783348,0.687450,0.647673,0.938452,0.666037,0.772563,1578,0.776321


#### Number of users

In [ ]:
print(f"number of users: {train_set.user_id.unique().size}")
print(f"number of videos: {train_set.video_id.unique().size}")

number of users: 7176
number of videos: 10728


In [ ]:
train_set.dtypes

user_id          int64
video_id         int64
time            object
watch_ratio    float64
dtype: object

In [ ]:
train_set.time = pd.to_datetime(train_set.time)
train_set.dtypes

user_id                 int64
video_id                int64
time           datetime64[ns]
watch_ratio           float64
dtype: object

In [ ]:
print(f"From {train_set.time.min()} to {train_set.time.max()}")
print(f"Duration: {train_set.time.max() - train_set.time.min()}")

From 2020-06-23 08:34:11.373000 to 2020-09-10 07:32:12.427000
Duration: 78 days 22:58:01.054000
